# Conditioned LSTM

In [339]:
import random

import numpy as np

import torch
from torch import nn
import torch.nn.functional as F

from sklearn.decomposition import PCA

import gensim
from gensim.models import KeyedVectors
from gensim.models.lsimodel import LsiModel
from gensim.corpora.dictionary import Dictionary

In [294]:
with open('data/tokenized.txt','r') as f:
    tokenized = eval(f.read())
    
with open('data/tokens.txt','r') as f:
    tokens = eval(f.read())

In [381]:
x_arr = np.load('data/x.npy')
y_arr = np.load('data/y.npy')

In [314]:
def get_batches(arr_x, arr_y, batch_size):
         
    # iterate through the arrays
    prv = 0
    for n in range(batch_size, arr_x.shape[0], batch_size):
        x = arr_x[prv:n]
        y = arr_y[prv:n]
        prv = n
        yield x, y

## Word2Vec

In [635]:
w2v_model = gensim.models.KeyedVectors.load("w2v.model", mmap='r')

In [651]:
vocab_size, emdedding_size = w2v_model.wv.vectors.shape
vocab_size, emdedding_size

(9569, 100)

In [653]:
embedding_size = w2v_model.wv.vectors.shape[1]
embedding_size

100

In [331]:
w2v_tensors = torch.FloatTensor(w2v_model.wv.vectors)

## Topic Modelling

### Corpus

In [302]:
dct = Dictionary(tokenized)
dct.filter_extremes(no_below=5, no_above=0.3)

In [303]:
len(dct)

2520

In [304]:
print(dct)

Dictionary(2520 unique tokens: ['across', 'all', 'annotation', 'arabic', 'baselines']...)


In [305]:
corpus = [dct.doc2bow(a) for a in tokenized]

### Latent Semantic Analysis (LSA)

In [308]:
n_hidden = 256
batch_size = 64
n_layers = 3

In [349]:
lsi = LsiModel(corpus, id2word=dct, num_topics=n_hidden, decay=0.2)

In [355]:
lsi.show_topics(5)

[(0,
  '-0.863*"e" + -0.298*"de" + -0.161*"d" + -0.139*"des" + -0.119*"la" + -0.101*"l" + -0.098*"les" + -0.093*"r" + -0.091*"et" + -0.088*"s"'),
 (1,
  '0.225*"word" + 0.157*"using" + 0.135*"it" + 0.133*"text" + 0.126*"information" + 0.122*"performance" + 0.121*"languages" + 0.120*"these" + 0.119*"or" + 0.117*"learning"'),
 (2,
  '-0.672*"word" + -0.259*"prediction" + -0.170*"speech" + -0.158*"part" + -0.138*"sequence" + -0.136*"words" + 0.130*"text" + -0.119*"using" + 0.092*"dataset" + 0.092*"domain"'),
 (3,
  '-0.541*"translation" + -0.311*"languages" + -0.279*"english" + -0.220*"machine" + -0.194*"corpus" + 0.160*"knowledge" + 0.141*"information" + -0.133*"resource" + -0.125*"mt" + -0.102*"parallel"'),
 (4,
  '0.258*"word" + 0.218*"translation" + -0.200*"corpus" + 0.175*"neural" + 0.156*"embeddings" + 0.151*"sentence" + -0.146*"system" + -0.143*"speech" + 0.141*"propose" + -0.131*"it"')]

In [357]:
lsi.show_topic(3, topn=10)

[('translation', -0.5411655517024783),
 ('languages', -0.3107445613281035),
 ('english', -0.2794159372094953),
 ('machine', -0.2201444291980297),
 ('corpus', -0.19399023760911902),
 ('knowledge', 0.16026022988917166),
 ('information', 0.1414960334516803),
 ('resource', -0.1326899685309229),
 ('mt', -0.12461609349835423),
 ('parallel', -0.1019574423580465)]

In [358]:
trans_topics = np.transpose(lsi.projection.u)

### Principal Component Analysis (PCA)

In [359]:
pca_topics = PCA(n_components=n_layers*batch_size, svd_solver='full').fit_transform(trans_topics)

In [360]:
pca_trans = np.transpose(pca_topics)

## Models

### Conditioned LSTM

In [667]:
class ConditionedLSTM(nn.Module):
    
    def __init__(self, n_hidden=256, n_layers=2, drop_prob=0.2, lr=0.001):
        super().__init__()

        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.emb_layer = nn.Embedding(vocab_size, embedding_size)

        # LSTM
        self.lstm = nn.LSTM(256, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        # fully-connected layer
        self.fc = nn.Linear(n_hidden, vocab_size)      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        
        x = x.long()

        # pass input through embedding layer
        embedded = self.emb_layer(x)     
        
        # get outputs and new hidden state from the lstm
        lstm_output, hidden = self.lstm(embedded, hidden)
        
        # pass through a dropout layer
        out = self.dropout(lstm_output)
        
        # flatten out
        out = out.reshape(-1, self.n_hidden) 

        # put "out" through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' initializes hidden state '''

        hidden = (torch.FloatTensor(pca_trans.reshape(self.n_layers, batch_size, self.n_hidden)),
                 torch.ones(self.n_layers, batch_size, self.n_hidden))

        return hidden

In [668]:
# instantiate the model
cond_lstm = ConditionedLSTM(n_hidden=n_hidden, n_layers=n_layers)

print(model)

ConditionedLSTM(
  (emb_layer): Embedding(9569, 100)
  (lstm): LSTM(100, 256, num_layers=3, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=256, out_features=9569, bias=True)
)


### Condtiioned LSTM + Word2Vec

In [661]:
class CondLSTM_Word2Vec(nn.Module):
    
    def __init__(self, n_hidden=256, n_layers=2, drop_prob=0.2, lr=0.001):
        super().__init__()

        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.emb_layer = nn.Embedding.from_pretrained(w2v_tensors)

        # LSTM
        self.lstm = nn.LSTM(256, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        # fully-connected layer
        self.fc = nn.Linear(n_hidden, vocab_size)      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        
        x = x.long()

        # pass input through embedding layer
        embedded = self.emb_layer(x)     
        
        # get outputs and new hidden state from the lstm
        lstm_output, hidden = self.lstm(embedded, hidden)
        
        # pass through a dropout layer
        out = self.dropout(lstm_output)
        
        # flatten out
        out = out.reshape(-1, self.n_hidden) 

        # put "out" through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' initializes hidden state '''

        hidden = (torch.FloatTensor(pca_trans.reshape(self.n_layers, batch_size, self.n_hidden)),
                 torch.ones(self.n_layers, batch_size, self.n_hidden))

        return hidden

In [662]:
# instantiate the model
cond_lstm_w2v = CondLSTM_Word2Vec(n_hidden=n_hidden, n_layers=n_layers)

print(model)

ConditionedLSTM(
  (emb_layer): Embedding(9569, 100)
  (lstm): LSTM(100, 256, num_layers=3, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=256, out_features=9569, bias=True)
)


In [663]:
def train(model, epochs=10, batch_size=32, lr=0.001, clip=1, print_every=32):
    
    # optimizer
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    
    # loss
    criterion = nn.CrossEntropyLoss()

    model.train()

    for e in range(epochs):
        
        batch = 0
        
        # initialize hidden state
        h = model.init_hidden(batch_size)
        
        for x, y in get_batches(x_arr, y_arr, batch_size):
            batch += 1
            
            # convert numpy arrays to PyTorch arrays
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            # detach hidden states
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            model.zero_grad()
            
            # get the output from the model
            output, h = model(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(-1).long())

            # back-propagate error
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)

            # update weigths
            opt.step()            
            
            if batch % print_every == 0:
                
                tot_batches = int(x_arr.shape[0] / batch_size)
            
                print("Epoch: {}/{} -".format(e+1, epochs),
                      "Batch: {}/{} -".format(batch, tot_batches),
                      "Loss: {}".format(loss))

In [664]:
epochs = 20
path = 'weights/cond_lstm.pt'
loss = 0.2

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [665]:
torch.save({
            'epoch': epochs,
            'model_state_dict': cond_lstm.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': criterion
            }, path)

In [666]:
train(cond_lstm, batch_size=batch_size, epochs=epochs, print_every=1)

Epoch: 1/20 - Batch: 1/22 - Loss: 6.7918219566345215
Epoch: 1/20 - Batch: 2/22 - Loss: 6.964037895202637
Epoch: 1/20 - Batch: 3/22 - Loss: 6.8836188316345215
Epoch: 1/20 - Batch: 4/22 - Loss: 6.85235595703125
Epoch: 1/20 - Batch: 5/22 - Loss: 6.844439506530762
Epoch: 1/20 - Batch: 6/22 - Loss: 6.944328784942627
Epoch: 1/20 - Batch: 7/22 - Loss: 7.489170551300049
Epoch: 1/20 - Batch: 8/22 - Loss: 6.902243137359619
Epoch: 1/20 - Batch: 9/22 - Loss: 7.010559558868408
Epoch: 1/20 - Batch: 10/22 - Loss: 6.920535087585449
Epoch: 1/20 - Batch: 11/22 - Loss: 6.931799411773682
Epoch: 1/20 - Batch: 12/22 - Loss: 6.935999870300293


KeyboardInterrupt: 

## Generation

In [396]:
def word_to_id(word):
    return w2v_model.wv.key_to_index[word]

def id_to_word(id):
    return w2v_model.wv.index_to_key[id]

In [397]:
# predict next token
def predict(model, t, h=None): # default value as None for first iteration
         
    # tensor inputs
    x = np.array([[word_to_id(t)]])
    inputs = torch.from_numpy(x)

    # detach hidden state from history
    h = tuple([each.data for each in h])

    # get the output of the model
    out, h = model(inputs, h)

    # get the token probabilities
    p = F.softmax(out, dim=1).data
    
    p = p.numpy()
    p = p.reshape(p.shape[1],)

    # get indices of top 3 values
    top_n_idx = p.argsort()[-5:][::-1]

    # randomly select one of the three indices
    sampled_token_index = top_n_idx[random.sample([0,1,2,3,4],1)[0]]

    # return the encoded value of the predicted char and the hidden state
    return id_to_word(sampled_token_index), h

In [398]:
gen_batch_size = 1

In [399]:
# PCA for generation batch size
gen_pca_topics = PCA(n_components=n_layers * gen_batch_size, svd_solver='full').fit_transform(trans_topics)
gen_pca_trans = np.transpose(gen_pca_topics)

In [400]:
# function to generate text
def generate(model=cond_lstm, words=10, prompt='in this paper'):
    
    model.eval()
    
    h = (torch.FloatTensor(gen_pca_trans.reshape(n_layers, gen_batch_size, n_hidden)),
         torch.ones(n_layers, gen_batch_size, n_hidden))

    toks = prompt.split()

    # predict next token
    for t in prompt.split():
        token, h = predict(model, t, h)
    
    toks.append(token)

    # predict subsequent tokens
    for i in range(words-1):
        token, h = predict(model, toks[-1], h)
        toks.append(token)

    return ' '.join(toks)

In [401]:
generate(words=20)

'in this paper efficient efficient efficient function rating function incorporates function rating incorporates implemented incorporates implemented more transformations transformations transformations that incorporates rating'

## Evaluation